In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
import yaml
import random
import json
import numpy as np
from copy import copy
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs

from collections import defaultdict
from probis import *
from run import *

In [3]:
with open("../cfg.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [4]:
pocket2rep_rec = get_rep_recs(cfg)
probis_scores = convert_inter_results_to_json(cfg)
smiles2filename = save_all_mol_sdfs(cfg)

Using cached data from get_rep_recs
Using cached data from convert_inter_results_to_json
Using cached data from save_all_mol_sdfs


In [5]:
big_clusters = get_clusters(cfg, pocket2rep_rec, probis_scores, z_cutoff=3.0)

In [6]:
val_df = pd.read_csv(cfg["bigbind_folder"] + "/activities_val.csv")

In [7]:
val_df["pchembl_value"] > 5.0

0        False
1         True
2        False
3         True
4         True
         ...  
88231    False
88232    False
88233     True
88234     True
88235     True
Name: pchembl_value, Length: 88236, dtype: bool

In [8]:
df = val_df
df["active"] = df["pchembl_value"] > 5.0

In [12]:
from collections import defaultdict

poc2cluster = {}
for cluster in big_clusters:
    for pocket in cluster:
        poc2cluster[pocket] = cluster

smiles2pockets = defaultdict(set)
for i, row in tqdm(df.iterrows(), total=len(df)):
    smiles2pockets[row.lig_smiles].add(row.pocket)

100%|██████████| 88236/88236 [00:09<00:00, 8955.08it/s] 


In [21]:
lens = []
for pocket in tqdm(df.pocket.unique()):
    lens.append((pocket, len(df.query("pocket == @pocket"))))

100%|██████████| 115/115 [00:00<00:00, 293.77it/s]


In [28]:
sorted(lens, key=lambda x: x[1])

[('PRTK_PARAQ_106_384_pep_0', 1),
 ('FAB1_MYCTU_1_416_0', 1),
 ('FOLB_STAAU_1_121_0', 1),
 ('SAV_STRAV_73_165_0', 1),
 ('VGP_EBOZM_34_298_0', 1),
 ('ARS_PSEAE_1_536_0', 2),
 ('BLAC_BACLI_42_306_0', 3),
 ('HEM2_ECOLI_2_324_0', 3),
 ('GGTA1_BOVIN_80_367_0', 4),
 ('ISPD_ARATH_75_302_0', 4),
 ('NR1I3_MOUSE_114_358_0', 5),
 ('TPIS_HUMAN_38_286_0', 5),
 ('SCMU_MYCTU_34_199_0', 5),
 ('ARGI1_RAT_4_321_0', 6),
 ('VNN1_HUMAN_22_513_0', 6),
 ('CPXA_PSEPU_2_415_0', 8),
 ('RISB_MYCTU_1_160_0', 8),
 ('UPP1_HUMAN_1_310_0', 9),
 ('DNMT1_MOUSE_730_1609_0', 13),
 ('RORB_RAT_214_465_ligBind_0', 15),
 ('HAOX1_HUMAN_1_370_0', 16),
 ('RNAS1_BOVIN_42_150_0', 19),
 ('GSTA1_HUMAN_1_222_0', 20),
 ('TP53B_HUMAN_1484_1603_0', 21),
 ('PP1G_HUMAN_2_314_ligBind_0', 22),
 ('AMD_RAT_498_820_0', 23),
 ('DNPH1_RAT_9_152_0', 24),
 ('DSBA_ECOLI_20_208_0', 24),
 ('CPNS1_HUMAN_93_267_0', 25),
 ('BLAB_BACFG_18_249_0', 27),
 ('PTGIS_HUMAN_20_500_0', 29),
 ('BXA1_CLOBO_2_425_0', 29),
 ('RICI_RICCO_36_233_0', 30),
 ('1433S_HUMA

In [29]:
act = df.query("pocket == 'VGFR2_HUMAN_808_1171_0'").active
sum(act)/len(act)

0.9532177630016623

In [30]:
def make_screen_df(cfg, df, pocket, smiles2pockets, tot_len=500, max_actives=50):
    poc_df = df.query("pocket == @pocket")
    active = poc_df.query("active")
    print(len(active), len(poc_df))

for pocket in df.pocket.unique():
    make_screen_df(cfg, df, pocket, smiles2pockets)

2080 2275
1781 1991
396 401
104 115
3 3
3946 4815
21 25
423 472
903 915
6184 6441
4364 4508
153 154
3 5
7 27
134 137
1824 1837
724 1389
4 8
79 93
260 376
860 945
1675 1752
4156 4476
30 57
58 58
8028 8422
136 140
15 16
14 20
83 110
1700 1908
410 485
2460 2528
273 330
1368 1412
5902 15966
132 149
220 227
22 23
538 578
14 29
211 217
21 22
584 742
396 440
84 126
4 5
1185 1210
3507 3576
222 232
569 609
1780 1835
2969 3139
192 195
38 47
680 758
91 96
149 155
1043 1052
253 334
550 681
1 5
74 99
82 116
23 52
81 99
6 19
22 33
17 29
121 139
30 40
1 2
464 604
101 103
212 218
334 340
180 186
669 672
0 1
182 253
11 47
1 8
458 584
45 76
9 9
62 89
21 46
655 655
194 194
301 328
1 4
4 30
753 797
1 1
45 65
4 4
250 289
7 21
1 1
329 340
0 3
37 37
6 13
20 24
1 1
14 24
85 102
17 34
78 82
63 99
5 6
4 15
88 102
1 1
6 6
